In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print(path)
        dataset = pd.read_csv(path)

/kaggle/input/dataset3/LREC_EmoInHindi.csv


In [2]:
!pip install torch
!pip install transformers

In [3]:
data = dataset[:10000]
data.head()

,dialogueId,utterance_no,authorRole,utterance,emotions,emoIntensity
0,0,0,bot,"नमस्ते, सुरक्षित वातावरण में आपकी मदद करने के ...","confident,anticipation","1,1"
1,0,1,user,पिछले शुक्रवार को तीन नकाबपोशों ने मेरी पत्नी ...,"sad,anger","3,3"
2,0,2,bot,"इतना घिनौना, क्या मैं जान सकता हूँ कि मैं किसक...","sad,anticipation","3,1"
3,0,3,user,मैं हूं अरुण सिन्हा। मैं चाहता हूं कि उन राक्ष...,"anger,confident","3,2"
4,0,4,bot,"ठीक है, अरुण सिन्हा, कृपया मुझे घटना की तारीख,...",anticipation,1
...,...,...,...,...,...,...
9995,1137,18,bot,कृपया किसी भी समय रक्षक से बेझिझक संपर्क करें।...,joy,1
9996,1138,0,bot,"नमस्ते, यह एक सुरक्षित वातावरण में आपकी सहायता...",confident,2
9997,1138,1,user,मेरा पति मेरी छोटी बेटी को मारने की योजना बना ...,anger,2
9998,1138,2,bot,यह सुनकर वाकई निराशा होती है। मैं आपके लिए यहा...,anticipation,2


In [4]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def calculate_metrics(y_true, y_pred, y_intensity_true, y_intensity_pred):
    emotion_accuracy = accuracy_score(y_true, y_pred)
    emotion_recall = recall_score(y_true, y_pred, average='macro')  
    emotion_precision = precision_score(y_true, y_pred, average='macro')
    emotion_f1 = f1_score(y_true, y_pred, average='macro')

    intensity_accuracy = accuracy_score(y_intensity_true, y_intensity_pred)
    intensity_recall = recall_score(y_intensity_true, y_intensity_pred, average='macro') 
    intensity_precision = precision_score(y_intensity_true, y_intensity_pred, average='macro')
    intensity_f1 = f1_score(y_intensity_true, y_intensity_pred, average='macro')

    return {
        'emotion_accuracy': emotion_accuracy,
        'emotion_recall': emotion_recall,
        'emotion_precision': emotion_precision,
        'emotion_f1': emotion_f1,
        'intensity_accuracy': intensity_accuracy,
        'intensity_recall': intensity_recall,
        'intensity_precision': intensity_precision,
        'intensity_f1': intensity_f1
    }
# y_pred = [1, 0, 0] 
# y_true = [1, 0, 0]
# y_intensity_true = [4, 2, 0] 
# y_intensity_pred = [4, 2, 0]
# result = calculate_metrics(y_true, y_pred, y_intensity_true, y_intensity_pred)
# print(result)

In [6]:
!git clone https://github.com/facebookresearch/fastText.git
!python /kaggle/working/fastText/download_model.py hi

import fasttext
ft = fasttext.load_model('/kaggle/working/cc.hi.300.bin')

fatal: destination path 'fastText' already exists and is not an empty directory.
File exists. Use --overwrite to download anyway.


In [7]:
def text_to_fasttext_embeddings(text):
    embeddings =  ft.get_word_vector(text)
    return embeddings

# temp = text_to_fasttext_embeddings("आज के समय में तकनीकी उन्नति ने हमारे जीवन को गहराई से प्रभावित किया है।")
# print(temp)

In [8]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

mapping = {
    'joy': 0,
    'anticipation': 1,
    'neutral': 2,
    'anger': 3,
    'disgusted': 4,
    'confident': 5,
    'annoyed': 6,
    'hopeful': 7,
    'apprehensive': 8,
    'grateful': 9,
    'sad': 10,
    'compassion': 11,
    'fear': 12,
    'guilty': 13,
    'surprised': 14,
    'impressed': 15,
    'confused': 16
}

def convert_to_float(embedding):
    return [float(value) for value in embedding]

def preprocess_data(data,batch_size):
    # Initialize lists for corpus, emotions, and intensities
    corpus = []
    emotions = []
    intensities = []

    # Iterate through the DataFrame
    for index, row in data.iterrows():
        # Extract utterance
        utterance = row['utterance']
        corpus.append(utterance)

        # Extract emotions and intensities
        emo = row['emotions'].split(',')
        emo_intensities = row['emoIntensity'].split(',')

        utterance_emotions = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        utterance_intensities = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        for idx, intensity in zip(emo,emo_intensities):
            utterance_emotions[mapping[idx]] = 1
            utterance_intensities[mapping[idx]] = int(intensity)

        emotions.append(utterance_emotions)
        intensities.append(utterance_intensities)
    
    text = [text_to_fasttext_embeddings(sentence) for sentence in corpus]
    # Convert text embeddings to float
    text_float = [convert_to_float(embedding) for embedding in text]

    # Convert data to PyTorch tensors
    text_tensor = torch.tensor(text_float, dtype=torch.float32)
    emotions_tensor = torch.tensor(emotions, dtype=torch.float32)
    intensities_tensor = torch.tensor(intensities, dtype=torch.float32)

    # Creating a TensorDataset
    dataset = TensorDataset(text_tensor, emotions_tensor, intensities_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    
    return dataloader

In [9]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

train_dataloader = preprocess_data(train_df,batch_size=2)
test_dataloader = preprocess_data(test_df,batch_size=2)
val_dataloader = preprocess_data(val_df,batch_size=2)

In [11]:
import torch
from torch import nn

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
      
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FNetBlock(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.unsqueeze(0) if x.dim() == 1 else x      #extra added
        x = torch.fft.fft(torch.fft.fft(x, dim=-1), dim=-2).real
        return x.squeeze(0) if x.dim() == 3 else x     #extra added
#         return x

class FNet(nn.Module):
    def __init__(self, dim, depth, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, FNetBlock()),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

In [12]:
#  # Model
class FNetEmotionClassifier(nn.Module):
    def __init__(self, embedding_dim=300, num_emotions=17, intensity_classes=17):
        super().__init__()

        self.fnet = FNet(dim=embedding_dim, depth=2, mlp_dim=embedding_dim * 2, dropout=0.3)  

        self.emotion_classifier = nn.Sequential(
            nn.Linear(embedding_dim, num_emotions),
            nn.Softmax(dim=-1)  # Softmax for multi-label emotion output
        )

        self.intensity_classifier = nn.Sequential(
            nn.Linear(embedding_dim, intensity_classes),
            nn.LogSoftmax(dim=-1)  # LogSoftmax for intensity classification
        )

    def forward(self, input_ids):
        fnet_output = self.fnet(input_ids)
#         print("fnet_output in cls FNetEmotionClassifier: ",fnet_output)
        emotion_logits = self.emotion_classifier(fnet_output)
        intensity_logits = self.intensity_classifier(fnet_output)
        return emotion_logits, intensity_logits 

In [13]:
import torch

class NLLLossCustom:
    def __init__(self):
        pass
    
    def __call__(self, intensity_logits, intensities):
        # Compute log probabilities
        log_probs = torch.log_softmax(intensity_logits, dim=1)
        
        # Reshape intensities to [batch_size * num_classes]
        num_samples, num_classes = log_probs.size()
        intensities = intensities.view(num_samples, num_classes)
        
        # Compute the negative log likelihood loss
        loss = -torch.sum(intensities * log_probs) / num_samples
        
        return loss


In [14]:
def aligne_predictions(emotions,emotion_logits,intensities,intensity_logits):
    """ used to align the predictions and the real values for the metric calculations"""
    emotions_actual = emotions.tolist()
    emotions_predicted = np.array(emotion_logits.tolist())
    intensity_predicted = intensity_logits.tolist()
    intensity_actual = intensities.tolist()

    for current in range(len(emotions_actual)):
        indices = [index for index, value in enumerate(emotions_actual[current]) if value ==1.0]
        emotion_true = indices
        emotion_true.sort()

        k = len(indices)
        emotion_pred = np.argsort(emotions_predicted[current])[-k:][::-1]
        emotion_pred.sort()

        intensity_pred = [int(intensity_predicted[current][index]) for index in indices]
        intensity_true = [int(intensity_actual[current][index]) for index in indices]

#         print("emotion_pred: ",emotion_pred)
#         print("emotion_true: ",emotion_true)
#         print("intensity_pred: ",intensity_pred)
#         print("intensity_true: ",intensity_true)
        
        return emotion_pred,emotion_true,intensity_pred,intensity_true

In [15]:
!export CUDA_LAUNCH_BLOCKING=1

model = FNetEmotionClassifier(num_emotions=17, intensity_classes=17)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn_emotions = nn.MultiLabelSoftMarginLoss()
# loss_fn_intensity = nn.NLLLoss()
loss_fn_intensity = NLLLossCustom()

In [16]:
import time
def train(model, dataloader, optimizer, loss_fn_emotions, loss_fn_intensity, device):
    model.train()
    total_loss = 0.0
    all_emotion_preds, all_emotion_true = [], []
    all_intensity_preds, all_intensity_true = [], []

    for batch in dataloader:
        start_time = time.time()  # Capture start time for the epoch 

        input_ids = batch[0].to(device)
        emotions = batch[1].to(device)  # Assuming labels are properly encoded
        intensities = batch[2].to(device)

        optimizer.zero_grad()
        emotion_logits, intensity_logits = model(input_ids)
#         print("intensity_logits: ",intensity_logits.shape)
#         print("intensities: ",intensities.shape)
        num_samples = emotion_logits.size(0)  # Get the number of samples
        num_classes = intensity_logits.size(1)  # Get the number of classes
        intensities = intensities.view(num_samples, num_classes)
            
        loss_emotion = loss_fn_emotions(emotion_logits, emotions)
        loss_intensity = loss_fn_intensity(intensity_logits, intensities)
        total_loss = loss_emotion + loss_intensity

        emotion_pred,emotion_true,intensity_pred,intensity_true = aligne_predictions(
                emotions,emotion_logits,intensities,intensity_logits)
            
        all_emotion_preds.extend(emotion_pred)
        all_emotion_true.extend(emotion_true)
        all_intensity_preds.extend(intensity_pred)
        all_intensity_true.extend(intensity_true)
            
        total_loss.backward()
        optimizer.step()

        # Gather predictions and true labels
#         all_emotion_preds.extend(emotion_logits.argmax(dim=1).tolist())
#         all_emotion_true.extend(emotions.tolist())  
#         all_intensity_preds.extend(intensity_logits.argmax(dim=1).tolist())
#         all_intensity_true.extend(intensities.tolist())

    epoch_metrics = calculate_metrics(all_emotion_true, all_emotion_preds,
                                      all_intensity_true, all_intensity_preds)
    epoch_time = time.time() - start_time

#     print(f"Epoch Training Loss: {total_loss:.3f}, Time: {epoch_time:.2f}s")
    print("="*60)
    print(f"Epoch Training Loss: {total_loss:.3f}")
    print(f"Epoch Time: {epoch_time:.2f}")
    print("="*60)
    
#     print("Epoch Metrics:")
#     for key, value in epoch_metrics.items():
#         print(f"{key}: {value:.2f}")

In [17]:
def evaluate(model, dataloader, loss_fn_emotions, loss_fn_intensity, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    all_emotion_preds, all_emotion_true = [], []
    all_intensity_preds, all_intensity_true = [], []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for batch in dataloader:
            input_ids = batch[0].to(device)   # input_ids
            emotions = batch[1].to(device)    # emotions
            intensities = batch[2].to(device)    # intensities

            emotion_logits, intensity_logits = model(input_ids)
            
            num_samples = emotion_logits.size(0)  # Get the number of samples
            num_classes = intensity_logits.size(1)  # Get the number of classes
            intensities = intensities.view(num_samples, num_classes)
#             intensities = intensities.argmax(dim=1)
#             print("emotion_logits: ",emotion_logits)
#             print("intensity_logits: ",intensity_logits)
#             print("intensities: ",intensities)
            
            loss_emotion = loss_fn_emotions(emotion_logits, emotions)
#             print("loss_emotion: ",loss_emotion)
            loss_intensity = loss_fn_intensity(intensity_logits, intensities)
#             print("loss_intensity: ",loss_intensity)
            total_loss += (loss_emotion + loss_intensity)
            emotion_pred,emotion_true,intensity_pred,intensity_true = aligne_predictions(
                emotions,emotion_logits,intensities,intensity_logits)
            
            all_emotion_preds.extend(emotion_pred)
            all_emotion_true.extend(emotion_true)
            all_intensity_preds.extend(intensity_pred)
            all_intensity_true.extend(intensity_true)
            
    metrics = calculate_metrics(all_emotion_true, all_emotion_preds,
                                all_intensity_true, all_intensity_preds)
    return total_loss / len(dataloader), metrics  # Average loss over the validation set


In [18]:
for epoch in range(20):
    train(model, train_dataloader, optimizer, loss_fn_emotions, loss_fn_intensity, device)   #change dataloader
    val_loss, val_metrics = evaluate(model, val_dataloader, loss_fn_emotions, loss_fn_intensity, device)
    print(f"Epoch {epoch+1} Validation Loss: {val_loss:.3f}, Metrics: {val_metrics} \n") 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch Training Loss: 0.700
Epoch Time: 0.06


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1 Validation Loss: 23.496, Metrics: {'emotion_accuracy': 0.2845360824742268, 'emotion_recall': 0.09689301276963329, 'emotion_precision': 0.06916741142285057, 'emotion_f1': 0.07544295383859406, 'intensity_accuracy': 0.0, 'intensity_recall': 0.0, 'intensity_precision': 0.0, 'intensity_f1': 0.0} 



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 19 Validation Loss: 5.281, Metrics: {'emotion_accuracy': 0.40103092783505156, 'emotion_recall': 0.16632593837960943, 'emotion_precision': 0.1668148745089085, 'emotion_f1': 0.1514350964100888, 'intensity_accuracy': 0.0, 'intensity_recall': 0.0, 'intensity_precision': 0.0, 'intensity_f1': 0.0} 



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch Training Loss: 3.466
Epoch Time: 0.05
Epoch 20 Validation Loss: 4.973, Metrics: {'emotion_accuracy': 0.4509394572025052, 'emotion_recall': 0.17947098764389005, 'emotion_precision': 0.17133421634465512, 'emotion_f1': 0.164136362993933, 'intensity_accuracy': 0.0, 'intensity_recall': 0.0, 'intensity_precision': 0.0, 'intensity_f1': 0.0} 



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
average_loss, metrics = evaluate(model, test_dataloader, loss_fn_emotions, loss_fn_intensity, device)
print("average_loss: ",average_loss)
print("metrics: ", metrics)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is i

average_loss:  tensor(4.6974, device='cuda:0')
metrics:  {'emotion_accuracy': 0.47822374039282667, 'emotion_recall': 0.1935358241464496, 'emotion_precision': 0.2872913096049109, 'emotion_f1': 0.1863721805300661, 'intensity_accuracy': 0.0, 'intensity_recall': 0.0, 'intensity_precision': 0.0, 'intensity_f1': 0.0}
